# uga 8360 p1 

In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
import numpy as np

In [4]:
import math

# initial setup of the spark

In [5]:
conf = SparkConf().setAppName("FirstNotebook")
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession.builder \
    .master("local") \
    .appName("Word Count") \
    .getOrCreate()

#  the learning nb based Dr. Quinn statement

In [8]:
# naive bayes training to compute

def naivebayeslearn():
    #feature number and class number are fixed and hard coded
    
    #1. each class probability
    bigp=[]
    
    #labels=sc.textFile("gs://uga-dsp/project1/files/y_train.txt").map(lambda x:(x,1)).reduceByKey(lambda x, y: x + y).sortByKey().collect()
    #total=sc.textFile("gs://uga-dsp/project1/files/y_train.txt").count()
    labels=sc.textFile("gs://uga-dsp/project1/files/y_small_train.txt").map(lambda x:(x,1)).reduceByKey(lambda x, y: x + y).sortByKey().collect()
    total=sc.textFile("gs://uga-dsp/project1/files/y_small_train.txt").count()
     
    
    
    
    
    
    
     
    
    for dic in labels:
        bigp.append(dic[1]/total)
        
        
    #2. each word counts of one class C1 , and of all class C2    
    featurep=np.ones((9,256))   #one could take care those 0 thing   
    #labelInOrder=sc.textFile("gs://uga-dsp/project1/files/y_train.txt").collect()    
    labelInOrder=sc.textFile("gs://uga-dsp/project1/files/y_small_train.txt").collect() 
    filenames=sc.textFile("gs://uga-dsp/project1/files/X_small_train.txt").collect() 
    
    
    #filenames=sc.textFile("gs://uga-dsp/project1/files/X_train.txt").collect()  
    
    commonpath="gs://uga-dsp/project1/data/bytes/"
    print('learning')
    aggregate=sc.emptyRDD() 

    for classindex in range(9):
        print("start ",classindex)
        initial=0 
        for file,indice in zip(filenames,labelInOrder):
            if (int(indice)-1)==classindex:          
                single=commonpath+file+'.bytes'
                #print(single)
                if initial==0:

                    t1=sc.textFile(single).flatMap(lambda line: line.split()[1:]) #remove the pointer and split each feature
                    assembly=t1.map(lambda x:(x,1)).reduceByKey(lambda x, y: x + y).sortByKey() #convert and aggregate
                    #print('this initial',end='\r')
                    #print('this initial: ',single, assembly.first())
                    initial=1
                    #aggregate=t1
                    aggregate=assembly
                else:
                    tw=sc.textFile(single).flatMap(lambda line: line.split()[1:]) #remove the pointer and split each feature
                    #aggregate=tw.union(aggregate)
                    #print("before combine",aggregate.first())  
                    #aggregate=aggregate.join(tw)
                    
                    assembly2=tw.map(lambda x:(x,1)).reduceByKey(lambda x, y: x + y).sortByKey() #convert and aggregate
                    #print(" the new ",assembly2.first())

                    #print(aggregate.first()) 
                    #print(single)
                    #aggregate=assembly2.union(aggregate)
                    #aggregate=assembly2.join(aggregate)
                    #rdd1=assembly2.zipWithIndex().map(lambda (val, key): (key,val))
                    #rdd2=aggregate.zipWithIndex().map(lambda (val, key): (key,val))
                    aggregate=aggregate.union(assembly2).reduceByKey(lambda x, y: x + y).sortByKey()
                    #flatMap(lambda x,[(y,z)]:x,y+z)
                    #print("after combine",aggregate.first())    
     
                #aggregate=aggregate.map(lambda x:(x,1)).reduceByKey(lambda x, y: x + y).sortByKey()
                index=0
                #print(aggregate.first(), end='\r')
        for item in aggregate.collect():
            if index<256:
                featurep[classindex][index]+=item[1]

                index+=1
        print('done: ',classindex)
        aggregate=sc.emptyRDD()         
            #else:
                #print('what is going on')
                               
        
    #guess need to union all the same class numbers before assigning to a numpy array.    
    return bigp,featurep

# run 

In [ ]:
b1,b2=naivebayeslearn()

learning
start  0
done:  0
start  1
done:  1
start  2
done:  2
start  3
done:  3
start  4
done:  4
start  5
done:  5
start  6
done:  6
start  7
done:  7
start  8
done:  8


In [11]:
b2/=np.sum(b2,axis=0)

In [ ]:
b1

In [12]:
b2[1][87]

0.4933217794802869

# prediction

In [ ]:
def naivebayesprediction(b1,b2):
    
    prediction=[]
    
    commonpath="gs://uga-dsp/project1/data/bytes/"
    
    #hard code the path
    #
    #for file in sc.textFile("gs://uga-dsp/project1/files/X_test.txt").collect():
    for file in sc.textFile("gs://uga-dsp/project1/files/X_small_test.txt").collect():
        single=commonpath+file+'.bytes'
        t1=sc.textFile(single).flatMap(lambda line: line.split()[1:]) #remove the pointer and split each feature
        assembly=t1.map(lambda x:(x,1)).reduceByKey(lambda x, y: x + y).sortByKey().collect() #convert and aggregate
        possibility=np.zeros((9))
        index=0
        for zain in range(9):
            possibility[zain]=math.log(b1[zain])
            for item in assembly:
                if index<256:
                    possibility[zain]+=math.log(b2[zain][index])*item[1]
                    index+=1
                    
        prediction.append(np.argmax(possibility)+1)
        
    
    
    return prediction

# predict

In [ ]:
result=naivebayesprediction(b1,b2)

In [17]:
result[7:9]

[3, 3]

In [18]:
groundtruth=sc.textFile("gs://uga-dsp/project1/files/y_small_train.txt").collect() 

In [23]:
groundtruth[0]

'6'

In [24]:
result[0]

3

In [25]:
bingo=0
for k,j in zip(groundtruth,result):
    if int(k)==j:
        bingo+=1
#bingo=groundtruth==result
print("accuracy: ", bingo/len(groundtruth))


accuracy:  0.12137203166226913


# serialize

In [ ]:
with open('gs://dsp-pj1/y_test.txt', 'w') as f:
    for item in result:
        
        f.write("%s\n" % (item+1))

In [ ]:
with open('y_test.txt', 'w') as f:
    for item in result:
        it=item+1
        f.write("%s\n" % it)